In [1]:
#reference: PROJECT REPORT: H1B DISCLOSURE DATASET - University of Texas Arlington

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import string
import numpy as np
import datetime as dt

In [3]:
df = pd.read_csv("H-1B_Disclosure_Data_FY17.csv", delimiter=',', encoding="utf-8", 
                 dtype={"EMPLOYER_COUNTRY": str, "EMPLOYER_PROVINCE": str, "EMPLOYER_PHONE":object, "SOC_NAME": str})

/anaconda3/envs/lca_project/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (16,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head(n=5)

,CASE_NUMBER,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYMENT_START_DATE,EMPLOYMENT_END_DATE,EMPLOYER_NAME,EMPLOYER_BUSINESS_DBA,EMPLOYER_ADDRESS,...,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,PUBLIC_DISCLOSURE_LOCATION,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,ORIGINAL_CERT_DATE
0,I-200-16055-173457,CERTIFIED-WITHDRAWN,2/24/16,10/1/16,H-1B,8/10/16,8/10/19,DISCOVER PRODUCTS INC.,NaN,2500 LAKE COOK ROAD,...,N,N,NaN,Y,NaN,RIVERWOODS,LAKE,IL,60015,3/1/16
1,I-200-16064-557834,CERTIFIED-WITHDRAWN,3/4/16,10/1/16,H-1B,8/16/16,8/16/19,DFS SERVICES LLC,NaN,2500 LAKE COOK ROAD,...,N,N,NaN,Y,NaN,RIVERWOODS,LAKE,IL,60015,3/8/16
2,I-200-16063-996093,CERTIFIED-WITHDRAWN,3/10/16,10/1/16,H-1B,9/9/16,9/9/19,EASTBANC TECHNOLOGIES LLC,NaN,1211 31ST ST. NW,...,Y,N,Y,NaN,NaN,WASHINGTON,NaN,DC,20007,3/16/16
3,I-200-16272-196340,WITHDRAWN,9/28/16,10/1/16,H-1B,1/26/17,1/25/20,INFO SERVICES LLC,NaN,17177 NORTH LAUREL PARK DR,...,Y,N,Y,NaN,NaN,JERSEY CITY,HUDSON,NJ,07302,NaN
4,I-200-15053-636744,CERTIFIED-WITHDRAWN,2/22/15,10/2/16,H-1B,3/1/15,3/1/18,BB&T CORPORATION,NaN,223 WEST NASH STREET,...,N,N,NaN,Y,NaN,NEW YORK,NEW YORK,NY,10036,2/26/15


In [5]:
len(df.columns)

52

In [6]:
len(df)

624650

In [7]:
df.isnull().sum()

CASE_NUMBER                         0
CASE_STATUS                         0
CASE_SUBMITTED                      0
DECISION_DATE                       0
VISA_CLASS                          0
EMPLOYMENT_START_DATE              29
EMPLOYMENT_END_DATE                30
EMPLOYER_NAME                      56
EMPLOYER_BUSINESS_DBA          581380
EMPLOYER_ADDRESS                    7
EMPLOYER_CITY                      15
EMPLOYER_STATE                     18
EMPLOYER_POSTAL_CODE               18
EMPLOYER_COUNTRY                96507
EMPLOYER_PROVINCE              618534
EMPLOYER_PHONE                  96508
EMPLOYER_PHONE_EXT             596778
AGENT_REPRESENTING_EMPLOYER     96506
AGENT_ATTORNEY_NAME                 0
AGENT_ATTORNEY_CITY            273306
AGENT_ATTORNEY_STATE           288641
JOB_TITLE                           5
SOC_CODE                            2
SOC_NAME                            3
NAICS_CODE                          7
TOTAL_WORKERS                       0
NEW_EMPLOYME

In [8]:
# The attributes with unique values are Case_Number, Employer_Address, 
# Employer_Phone and Employer_Phone_Ext. Case_Number is a unique identifier that is
# assigned to each case. Employer address and phone number 
# are unnecessary attributes in predicting the Case_Status. 
# So, we removed unnecessary attributes Case_Number, Employer_Address, 
# Employer_Phone and Employer_Phone_Ext because they have unique values.

df = df.drop(columns=['CASE_NUMBER', 'EMPLOYER_ADDRESS', 'EMPLOYER_PHONE', 'EMPLOYER_PHONE_EXT'])

In [9]:
# We also had repetitive attributes such as Employer_City, Worksite_City, Worksite_County, 
# Employer_Province, Employer_Postal_Code that were providing the same information.
# Those attributes were removed as well. We kept Employer_State and Worksite_State 
# since there are 50 states and 5 US territories, we can analyze class variable based on that.

df = df.drop(columns=['EMPLOYER_CITY', 'EMPLOYER_COUNTRY', 'WORKSITE_CITY', 'WORKSITE_COUNTY', 
                 'EMPLOYER_PROVINCE', 'EMPLOYER_POSTAL_CODE', 'WORKSITE_POSTAL_CODE'])

In [10]:
# Job_Title, SOC_Code and SOC_Name were all providing the same occupational information. 
# So, we removed Job_Title and SOC_Code and kept SOC_Name.

df = df.drop(columns=['JOB_TITLE','SOC_CODE'])

In [11]:
# The following attributes had more than 80% of missing values so were removed:
# AGENT_ATTORNEY_NAME
# AGENT_ATTORNEY_CITY
# AGENT_ATTORNEY_STATE
# ORIGINAL_CERT_DATE

df = df.drop(columns=['AGENT_ATTORNEY_NAME','AGENT_ATTORNEY_CITY',
                 'AGENT_ATTORNEY_STATE', 'ORIGINAL_CERT_DATE'])

In [15]:
len(df)
len(df.columns)

35

In [24]:
df['EMPLOYMENT_START_DATE'] = pd.to_datetime(df['EMPLOYMENT_START_DATE'], format= '%m/%d/%y')
df['EMPLOYMENT_END_DATE'] = pd.to_datetime(df['EMPLOYMENT_END_DATE'], format= '%m/%d/%y')

In [26]:
# For Labor Condition Application, employment time cannot be more than three years. In
# our dataset there are two variables Employment_Start_Date and Employment_End_Date.
# We calculated the difference in time between those two attributes and more than 90% of
# the difference was 3 years for our dataset. That would make them irrelevant attributes in
# predicting our class variable, so they were removed.
timediff = df['EMPLOYMENT_END_DATE'] - df['EMPLOYMENT_START_DATE']
td = round(timediff / np.timedelta64(1, 'Y'), 1)
td3 = td[td>=3]

In [41]:
len(td3)/len(td)

0.8987240854878732

In [42]:
df = df.drop(columns=['EMPLOYMENT_START_DATE','EMPLOYMENT_END_DATE'])

In [43]:
df.head(5)

,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,VISA_CLASS,EMPLOYER_NAME,EMPLOYER_BUSINESS_DBA,EMPLOYER_STATE,AGENT_REPRESENTING_EMPLOYER,SOC_NAME,NAICS_CODE,...,PW_SOURCE_OTHER,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,H1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,LABOR_CON_AGREE,PUBLIC_DISCLOSURE_LOCATION,WORKSITE_STATE
0,CERTIFIED-WITHDRAWN,2/24/16,10/1/16,H-1B,DISCOVER PRODUCTS INC.,NaN,IL,Y,COMPUTER SYSTEMS ANALYSTS,522210,...,OFLC ONLINE DATA CENTER,"65,811.00","67,320.00",Year,N,N,NaN,Y,NaN,IL
1,CERTIFIED-WITHDRAWN,3/4/16,10/1/16,H-1B,DFS SERVICES LLC,NaN,IL,Y,OPERATIONS RESEARCH ANALYSTS,522210,...,TOWERS WATSON DATA SERVICES 2015 CSR PROFESSIO...,"53,000.00","57,200.00",Year,N,N,NaN,Y,NaN,IL
2,CERTIFIED-WITHDRAWN,3/10/16,10/1/16,H-1B,EASTBANC TECHNOLOGIES LLC,NaN,DC,Y,COMPUTER PROGRAMMERS,541511,...,OFLC ONLINE DATA CENTER,"77,000.00",0.00,Year,Y,N,Y,NaN,NaN,DC
3,WITHDRAWN,9/28/16,10/1/16,H-1B,INFO SERVICES LLC,NaN,MI,N,"COMPUTER OCCUPATIONS, ALL OTHER",541511,...,OFLC ONLINE DATA CENTER,"102,000.00",0.00,Year,Y,N,Y,NaN,NaN,NJ
4,CERTIFIED-WITHDRAWN,2/22/15,10/2/16,H-1B,BB&T CORPORATION,NaN,NC,Y,CREDIT ANALYSTS,522110,...,OFLC ONLINE DATA CENTER,"132,500.00",0.00,Year,N,N,NaN,Y,NaN,NY
